In [1]:
import pandas  as pd 
import numpy as np 

In [3]:
df = pd.read_csv(r'C:\Users\Yibabe\Desktop\ball_bearing\data\LogFile_2022-06-20-17-00-31.csv') 

In [4]:

# Split the DataFrame into 10 equal parts
df_parts = np.array_split(df, 10)

# Save each part as a separate CSV file (optional)
for i, part in enumerate(df_parts, 1):
    part.to_csv(f'C:\\Users\\Yibabe\\Desktop\\ball_bearing\\data\\LogFile_part_{i}.csv', index=False)

# If you want to access each part separately, you can do:
df1, df2, df3, df4, df5, df6, df7, df8, df9, df10 = df_parts


c:\Users\Yibabe\Desktop\ball_bearing\venv\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [2]:
df1 = pd.read_csv(r"C:\Users\Yibabe\Desktop\ball_bearing\data\LogFile_part_1.csv")

In [3]:
# Assign proper column names
columns = ['x_direction', 'y_direction', 'T_bearing', 'T_env']
df1.columns = columns

In [19]:
df1.head()

,x_direction,y_direction,T_bearing,T_env
0,-0.117601,-0.378231,41.614912,24.817354
1,-0.125842,-0.447849,41.614912,24.817354
2,0.174730,-0.201366,41.614912,24.817354
3,0.298664,-0.093859,41.614912,24.817354
4,0.235976,-0.049856,41.614912,24.817354


In [4]:
df1 = df1.drop(['T_bearing'],axis = 1)

In [5]:
df1 =df1.drop(['T_env'], axis =1)

In [6]:
# Example dataframe with 200,000 rows (Replace this with your actual data)
df1= pd.DataFrame({'x_direction': np.random.randn(200000), 'y_direction': np.random.randn(200000)})

In [7]:

# Total number of rows
total_rows = len(df1)

In [8]:
# Compute elapsed hours
df1['elapsed_hours'] = (np.arange(total_rows) / total_rows) * 128  

In [9]:
# Compute RUL
df1['RUL'] = 128 - df1['elapsed_hours']


In [10]:
print(df1.head())  
print(df1.tail())  

   x_direction  y_direction  elapsed_hours        RUL
0     0.143240     1.440019        0.00000  128.00000
1    -0.286841    -1.197290        0.00064  127.99936
2    -0.919440     1.001554        0.00128  127.99872
3    -0.041799     1.213839        0.00192  127.99808
4    -0.082392     0.582228        0.00256  127.99744
        x_direction  y_direction  elapsed_hours      RUL
199995    -2.129877    -0.352677      127.99680  0.00320
199996    -0.277118    -0.734965      127.99744  0.00256
199997     1.881860    -0.255821      127.99808  0.00192
199998    -0.559205     1.012283      127.99872  0.00128
199999     0.347295    -0.648222      127.99936  0.00064


In [12]:
df1.isnull().sum()

x_direction      0
y_direction      0
elapsed_hours    0
RUL              0
dtype: int64

Normalization Method
We'll use MinMaxScaler from sklearn to scale features between 0 and 1 

In [13]:
from sklearn.preprocessing import MinMaxScaler
# Select features to normalize
scaler = MinMaxScaler()
# Fit and transform x_direction, y_direction, and RUL
df1[['x_direction', 'y_direction', 'RUL']] = scaler.fit_transform(df1[['x_direction', 'y_direction', 'RUL']])

In [14]:
# Show sample normalized data
print(df1.head(5))
print(df1.tail(5))

   x_direction  y_direction  elapsed_hours       RUL
0     0.497728     0.620714        0.00000  1.000000
1     0.452512     0.328217        0.00064  0.999995
2     0.386005     0.572085        0.00128  0.999990
3     0.478275     0.595629        0.00192  0.999985
4     0.474007     0.525579        0.00256  0.999980
        x_direction  y_direction  elapsed_hours       RUL
199995     0.258748     0.421891      127.99680  0.000020
199996     0.453535     0.379492      127.99744  0.000015
199997     0.680515     0.432633      127.99808  0.000010
199998     0.423878     0.573275      127.99872  0.000005
199999     0.519181     0.389113      127.99936  0.000000


In [15]:
def create_sequences(df1, time_steps=50):
    X, y = [], []
    for i in range(len(df1) - time_steps):
        X.append(df1[['x_direction', 'y_direction']].iloc[i:i+time_steps].values)
        y.append(df1['RUL'].iloc[i+time_steps])  # Predict RUL at the last step
    return np.array(X), np.array(y)

# Create sequences (choose time_steps = 50)
time_steps = 50
X, y = create_sequences(df1, time_steps)



In [16]:
# Check shape
print(f"X shape: {X.shape}")  # Expected: (num_samples, time_steps, features)
print(f"y shape: {y.shape}")  # Expected: (num_samples,)


X shape: (199950, 50, 2)
y shape: (199950,)


In [17]:
from sklearn.model_selection import train_test_split

# Define train-test split ratio (e.g., 80% train, 20% test)
train_size = int(0.8 * len(X))

# Split data
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]




In [18]:
# Print shapes
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train shape: (159960, 50, 2), y_train shape: (159960,)
X_test shape: (39990, 50, 2), y_test shape: (39990,)


In [19]:
from sklearn.preprocessing import MinMaxScaler

# Initialize scaler
scaler = MinMaxScaler()

# Reshape data into 2D format for scaling
X_train_reshaped = X_train.reshape(-1, 2)  # Convert (159960, 50, 2) → (7998000, 2)
X_test_reshaped = X_test.reshape(-1, 2)  # Convert (39990, 50, 2) → (1999500, 2)



7998000

In [23]:
# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train_reshaped)
X_test_scaled = scaler.transform(X_test_reshaped)  # Use the same scaler for test data



In [24]:
# Reshape back to original shape
X_train = X_train_scaled.reshape(X_train.shape)
X_test = X_test_scaled.reshape(X_test.shape)


In [27]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Define model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(50, 2)),  # First LSTM layer
    Dropout(0.2),  # Dropout to prevent overfitting
    LSTM(32, return_sequences=False),  # Second LSTM layer
    Dropout(0.2),
    Dense(16, activation='relu'),  # Fully connected layer
    Dense(1, activation='relu')  # Output layer (ensures RUL is non-negative)
])

# Compile model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Show summary
model.summary()


c:\Users\Yibabe\Desktop\ball_bearing\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 50, 64)         │        17,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 50, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,113 (117.63 KB)

 Trainable params: 30,113 (117.63 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
# Train the model
history = model.fit(
    X_train, y_train, 
    validation_data=(X_test, y_test), 
    epochs=20, 
    batch_size=64,
    verbose=1
)


Epoch 1/20
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 221s 81ms/step - loss: 0.0596 - mae: 0.2078 - val_loss: 0.2562 - val_mae: 0.5029
Epoch 2/20
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 276s 88ms/step - loss: 0.0535 - mae: 0.2002 - val_loss: 0.2475 - val_mae: 0.4941
Epoch 3/20
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 271s 91ms/step - loss: 0.0536 - mae: 0.2006 - val_loss: 0.2577 - val_mae: 0.5043
Epoch 4/20
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 267s 94ms/step - loss: 0.0533 - mae: 0.1998 - val_loss: 0.2617 - val_mae: 0.5083
Epoch 5/20
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 247s 99ms/step - loss: 0.0535 - mae: 0.2003 - val_loss: 0.2572 - val_mae: 0.5038
Epoch 6/20
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - loss: 0.0534 - mae: 0.2001